In [9]:
import MetaTrader5 as mt5
from datetime import datetime, timedelta, time
from IPython.display import display

from atj_algotrading.backtester import Backtester, get_ohlc_history, create_price_fig, evaluate_backtest

In [10]:
symbol = 'USDJPY'
start_dt = datetime(2023, 1, 1)
end_dt = datetime.now()

# get historical data
mt5.initialize()
ohlc = get_ohlc_history(symbol, mt5.TIMEFRAME_D1, start_dt, end_dt)

sma_period = 20

ohlc['sma_20'] = ohlc['open'].rolling(sma_period).mean()
ohlc['std'] = ohlc['open'].rolling(sma_period).std()
ohlc['upper_band'] = ohlc['sma_20'] + 1.5 * ohlc['std']
ohlc['lower_band'] = ohlc['sma_20'] - 1.5 * ohlc['std']


display(ohlc)

fig = create_price_fig(ohlc, indicators=['sma_20', 'upper_band', 'lower_band'])
display(fig)

,time,open,high,low,close,sma_20,std,upper_band,lower_band
0,2023-01-02,130.923,131.042,130.601,130.602,NaN,NaN,NaN,NaN
1,2023-01-03,130.728,131.407,129.517,130.985,NaN,NaN,NaN,NaN
2,2023-01-04,131.016,132.716,129.927,132.643,NaN,NaN,NaN,NaN
3,2023-01-05,132.509,134.055,131.684,133.422,NaN,NaN,NaN,NaN
4,2023-01-06,133.395,134.772,131.996,132.086,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
449,2024-09-24,143.453,144.681,143.110,143.214,143.27350,1.652646,145.752469,140.794531
450,2024-09-25,143.091,144.840,142.902,144.741,143.23270,1.646243,145.702065,140.763335
451,2024-09-26,144.703,145.212,144.110,144.797,143.24385,1.655863,145.727644,140.760056
452,2024-09-27,144.750,146.490,142.065,142.113,143.23765,1.649658,145.712137,140.763163


In [11]:
def get_signal(x):
    if x['open'] > x['upper_band']:
        return 'sell'
    elif x['open'] < x['lower_band']:
        return 'buy'
    
def get_exit_signal(x):
    if x['open'] > x['sma_20']:
        return 'close buy positions'
    elif x['open'] < x['lower_band']:
        return 'close sell positions'
    
ohlc['signal'] = ohlc.apply(get_signal, axis=1)
ohlc['exit_signal'] = ohlc.apply(get_exit_signal, axis=1)
ohlc

,time,open,high,low,close,sma_20,std,upper_band,lower_band,signal,exit_signal
0,2023-01-02,130.923,131.042,130.601,130.602,NaN,NaN,NaN,NaN,None,None
1,2023-01-03,130.728,131.407,129.517,130.985,NaN,NaN,NaN,NaN,None,None
2,2023-01-04,131.016,132.716,129.927,132.643,NaN,NaN,NaN,NaN,None,None
3,2023-01-05,132.509,134.055,131.684,133.422,NaN,NaN,NaN,NaN,None,None
4,2023-01-06,133.395,134.772,131.996,132.086,NaN,NaN,NaN,NaN,None,None
...,...,...,...,...,...,...,...,...,...,...,...
449,2024-09-24,143.453,144.681,143.110,143.214,143.27350,1.652646,145.752469,140.794531,None,close buy positions
450,2024-09-25,143.091,144.840,142.902,144.741,143.23270,1.646243,145.702065,140.763335,None,None
451,2024-09-26,144.703,145.212,144.110,144.797,143.24385,1.655863,145.727644,140.760056,None,close buy positions
452,2024-09-27,144.750,146.490,142.065,142.113,143.23765,1.649658,145.712137,140.763163,None,close buy positions


In [12]:
# create trade logic
def on_bar(data, trades, orders):
    volume = 100000 # 1 lots
    
    open_trades = trades[trades['state'] == 'open']
    num_open_trades = open_trades.shape[0]
    
    # entry signal
    if data['signal'] == 'buy' and not num_open_trades:
        orders.open_trade(symbol, volume, 'buy')
    
    elif data['signal'] == 'sell' and not num_open_trades:
        orders.open_trade(symbol, volume, 'sell')
        
    # exit signal
    if num_open_trades:
        trade = open_trades.iloc[0]

        if trade['order_type'] == 'buy' and data['exit_signal'] == 'close buy positions':
            orders.close_trade(trade)
        elif trade['order_type'] == 'sell' and data['exit_signal'] == 'close sell positions':
            orders.close_trade(trade)

In [13]:
# backtest parameters
starting_balance = 10000
currency = 'USD'
exchange_rate = 1/150
commission = -7 / 100000

# backtest
bt = Backtester()
bt.set_starting_balance(starting_balance, currency=currency)
bt.set_exchange_rate(exchange_rate)
bt.set_commission(commission)

bt.set_historical_data(ohlc)
bt.set_on_bar(on_bar)

bt.run_backtest()

bt.trades

,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,info,profit,commission,profit_net,profit_cumulative,balance
0,closed,USDJPY,sell,100000,2023-02-06 00:00:00,132.441,2023-03-14 00:00:00,133.175,0,0,None,-489.33,-7.0,-496.33,-496.33,9503.67
1,closed,USDJPY,buy,100000,2023-03-16 00:00:00,133.355,2023-04-03 00:00:00,133.339,0,0,None,-10.67,-7.0,-17.67,-514.0,9486.0
2,closed,USDJPY,sell,100000,2023-04-17 00:00:00,133.745,2023-07-12 00:00:00,140.284,0,0,None,-4359.33,-7.0,-4366.33,-4880.33,5119.67
3,closed,USDJPY,buy,100000,2023-07-13 00:00:00,138.413,2023-07-31 00:00:00,140.887,0,0,None,1649.33,-7.0,1642.33,-3238.0,6762.0
4,closed,USDJPY,sell,100000,2023-08-03 00:00:00,143.316,2023-11-21 00:00:00,148.338,0,0,None,-3348.00,-7.0,-3355.0,-6593.0,3407.0
5,closed,USDJPY,buy,100000,2023-11-22 00:00:00,148.299,2024-01-05 00:00:00,144.532,0,0,None,-2511.33,-7.0,-2518.33,-9111.33,888.67
6,closed,USDJPY,sell,100000,2024-01-11 00:00:00,145.606,2024-03-07 00:00:00,149.311,0,0,None,-2470.00,-7.0,-2477.0,-11588.33,-1588.33
7,closed,USDJPY,buy,100000,2024-03-08 00:00:00,147.967,2024-03-20 00:00:00,150.824,0,0,None,1904.67,-7.0,1897.67,-9690.66,309.34
8,closed,USDJPY,sell,100000,2024-04-11 00:00:00,153.121,2024-06-05 00:00:00,154.801,0,0,None,-1120.00,-7.0,-1127.0,-10817.66,-817.66
9,closed,USDJPY,sell,100000,2024-06-21 00:00:00,158.89,2024-07-18 00:00:00,156.125,0,0,None,1843.33,-7.0,1836.33,-8981.33,1018.67


In [14]:
pnl_chart = bt.plot_pnl()
pnl_chart

In [15]:
backtest_fig = bt.visualize_backtest(indicators=['sma_20', 'upper_band', 'lower_band'])
backtest_fig

In [16]:
evaluate_backtest(bt.trades)

biggest_profit 1904.67
biggest_loss -7550.67


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,info,profit,commission,profit_net,profit_cumulative,balance
3,closed,USDJPY,buy,100000,2023-07-13,138.413,2023-07-31,140.887,0,0,None,1649.33,-7.0,1642.33,-3238.0,6762.0
7,closed,USDJPY,buy,100000,2024-03-08,147.967,2024-03-20,150.824,0,0,None,1904.67,-7.0,1897.67,-9690.66,309.34
9,closed,USDJPY,sell,100000,2024-06-21,158.89,2024-07-18,156.125,0,0,None,1843.33,-7.0,1836.33,-8981.33,1018.67
11,closed,USDJPY,buy,100000,2024-09-05,143.611,2024-09-23,143.89,0,0,None,186.00,-7.0,179.0,-16360.0,-6360.0


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,info,profit,commission,profit_net,profit_cumulative,balance
0,closed,USDJPY,sell,100000,2023-02-06,132.441,2023-03-14,133.175,0,0,None,-489.33,-7.0,-496.33,-496.33,9503.67
1,closed,USDJPY,buy,100000,2023-03-16,133.355,2023-04-03,133.339,0,0,None,-10.67,-7.0,-17.67,-514.0,9486.0
2,closed,USDJPY,sell,100000,2023-04-17,133.745,2023-07-12,140.284,0,0,None,-4359.33,-7.0,-4366.33,-4880.33,5119.67
4,closed,USDJPY,sell,100000,2023-08-03,143.316,2023-11-21,148.338,0,0,None,-3348.00,-7.0,-3355.0,-6593.0,3407.0
5,closed,USDJPY,buy,100000,2023-11-22,148.299,2024-01-05,144.532,0,0,None,-2511.33,-7.0,-2518.33,-9111.33,888.67
6,closed,USDJPY,sell,100000,2024-01-11,145.606,2024-03-07,149.311,0,0,None,-2470.00,-7.0,-2477.0,-11588.33,-1588.33
8,closed,USDJPY,sell,100000,2024-04-11,153.121,2024-06-05,154.801,0,0,None,-1120.00,-7.0,-1127.0,-10817.66,-817.66
10,closed,USDJPY,buy,100000,2024-07-19,157.327,2024-09-02,146.001,0,0,None,-7550.67,-7.0,-7557.67,-16539.0,-6539.0


avg_win 1395.8325
avg_loss -2732.41625
count_profit_trades 4
count_loss_trades 8
win_rate 0.5
rrr 0.51


,order_type,profit
0,buy,-6332.67
1,sell,-9943.33


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,info,profit,commission,profit_net,profit_cumulative,balance,dayofweek
0,closed,USDJPY,sell,100000,2023-02-06,132.441,2023-03-14,133.175,0,0,None,-489.33,-7.0,-496.33,-496.33,9503.67,1
1,closed,USDJPY,buy,100000,2023-03-16,133.355,2023-04-03,133.339,0,0,None,-10.67,-7.0,-17.67,-514.0,9486.0,0
2,closed,USDJPY,sell,100000,2023-04-17,133.745,2023-07-12,140.284,0,0,None,-4359.33,-7.0,-4366.33,-4880.33,5119.67,2
3,closed,USDJPY,buy,100000,2023-07-13,138.413,2023-07-31,140.887,0,0,None,1649.33,-7.0,1642.33,-3238.0,6762.0,0
4,closed,USDJPY,sell,100000,2023-08-03,143.316,2023-11-21,148.338,0,0,None,-3348.00,-7.0,-3355.0,-6593.0,3407.0,1
5,closed,USDJPY,buy,100000,2023-11-22,148.299,2024-01-05,144.532,0,0,None,-2511.33,-7.0,-2518.33,-9111.33,888.67,4
6,closed,USDJPY,sell,100000,2024-01-11,145.606,2024-03-07,149.311,0,0,None,-2470.00,-7.0,-2477.0,-11588.33,-1588.33,3
7,closed,USDJPY,buy,100000,2024-03-08,147.967,2024-03-20,150.824,0,0,None,1904.67,-7.0,1897.67,-9690.66,309.34,2
8,closed,USDJPY,sell,100000,2024-04-11,153.121,2024-06-05,154.801,0,0,None,-1120.00,-7.0,-1127.0,-10817.66,-817.66,2
9,closed,USDJPY,sell,100000,2024-06-21,158.89,2024-07-18,156.125,0,0,None,1843.33,-7.0,1836.33,-8981.33,1018.67,3


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,info,profit,commission,profit_net,profit_cumulative,balance,dayofweek,current_max,drawdown
0,closed,USDJPY,sell,100000,2023-02-06,132.441,2023-03-14,133.175,0,0,None,-489.33,-7.0,-496.33,-496.33,9503.67,1,-496.33,0.0
1,closed,USDJPY,buy,100000,2023-03-16,133.355,2023-04-03,133.339,0,0,None,-10.67,-7.0,-17.67,-514.0,9486.0,0,-496.33,-17.67
2,closed,USDJPY,sell,100000,2023-04-17,133.745,2023-07-12,140.284,0,0,None,-4359.33,-7.0,-4366.33,-4880.33,5119.67,2,-496.33,-4384.0
3,closed,USDJPY,buy,100000,2023-07-13,138.413,2023-07-31,140.887,0,0,None,1649.33,-7.0,1642.33,-3238.0,6762.0,0,-496.33,-2741.67
4,closed,USDJPY,sell,100000,2023-08-03,143.316,2023-11-21,148.338,0,0,None,-3348.00,-7.0,-3355.0,-6593.0,3407.0,1,-496.33,-6096.67
5,closed,USDJPY,buy,100000,2023-11-22,148.299,2024-01-05,144.532,0,0,None,-2511.33,-7.0,-2518.33,-9111.33,888.67,4,-496.33,-8615.0
6,closed,USDJPY,sell,100000,2024-01-11,145.606,2024-03-07,149.311,0,0,None,-2470.00,-7.0,-2477.0,-11588.33,-1588.33,3,-496.33,-11092.0
7,closed,USDJPY,buy,100000,2024-03-08,147.967,2024-03-20,150.824,0,0,None,1904.67,-7.0,1897.67,-9690.66,309.34,2,-496.33,-9194.33
8,closed,USDJPY,sell,100000,2024-04-11,153.121,2024-06-05,154.801,0,0,None,-1120.00,-7.0,-1127.0,-10817.66,-817.66,2,-496.33,-10321.33
9,closed,USDJPY,sell,100000,2024-06-21,158.89,2024-07-18,156.125,0,0,None,1843.33,-7.0,1836.33,-8981.33,1018.67,3,-496.33,-8485.0


ValueError: Plotly Express cannot process wide-form data with columns of different type.